In [1]:
import os
import sys
print(os.getcwd())
sys.path.append("../WanPy")

import numpy as np
from pythtb import *
from wpythtb import Bloch, K_mesh, Wannier
from models import *
import matplotlib.pyplot as plt

/Users/treycole/Repos/WanPy/dev


In [2]:
t = 1.0  # spin-independent first-neighbor hop
soc = 0.125  # spin-dependent second-neighbor hop
m = 1  # magnetic field magnitude
beta = np.pi
h = m*np.sin(beta)*np.array([1, 1, 1])
dt = m*np.cos(beta)    # modification to t for (111) bond

fkm_model = fu_kane_mele(t, dt, soc, h)

nks = 20, 20, 20
bloch_states = Bloch(fkm_model, *nks)
bloch_states.solve_model()

In [ ]:
bloch_states.\

3